## 1.	Detection and recognition of objects (including human)
## 3.	Detection and recognition of human faces (using MTCNN, YOLO or whatever you would like to use)

#  1-1. 유튜브에서 v1파일 업로드

In [ ]:
#미리 pip install pytube 실행해야함.
from pytube import Playlist
p1 = Playlist('https://www.youtube.com/playlist?list=PL69GGs-Tg_mu-GV9k4t3jK-3hEvAQFtZn')#재생목록 url
p1.download_all('./videos')#저장위치

# 1-2.동영상에 yolo적용

In [ ]:
from imageai.Detection import VideoObjectDetection
import os

execution_path = os.getcwd()

detector = VideoObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath( os.path.join(execution_path , "models/yolo.h5"))
detector.loadModel()

video_path = detector.detectObjectsFromVideo(
    input_file_path=os.path.join(execution_path, "./videos/1V1-face.mp4"), #input+경로
    output_file_path=os.path.join(execution_path, "V1"), #output
    frames_per_second=20, log_progress=True)
print(video_path)

#  3-1. MTCNN으로 얼굴인식

In [ ]:
import cv2
import numpy as np
from mtcnn.mtcnn import MTCNN

detector = MTCNN()
video_file_to_save ="./videos/V1_saved.mp4"
cap = cv2.VideoCapture('./videos/1V1-face.mp4')

if cap.isOpened():
    fps = 25.40 # 초당프레임수
    fourcc = cv2.VideoWriter_fourcc(*'MPG4') # 인코딩포맷
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    size = (int(width), int(height)) # 프레임크기
    out = cv2.VideoWriter(video_file_to_save, fourcc, fps, size) #VideoWriter object
    while True: 
    
        ret, frame = cap.read()
    
        result = detector.detect_faces(frame)
        if result != []:
            for person in result:
                bounding_box = person['box']
                keypoints = person['keypoints']
    
                cv2.rectangle(frame,
                              (bounding_box[0], bounding_box[1]),
                              (bounding_box[0]+bounding_box[2], bounding_box[1] + bounding_box[3]),
                              (0,155,255),
                              2)
    
                cv2.circle(frame,(keypoints['left_eye']), 2, (0,155,255), 2)
                cv2.circle(frame,(keypoints['right_eye']), 2, (0,155,255), 2)
                cv2.circle(frame,(keypoints['nose']), 2, (0,155,255), 2)
                cv2.circle(frame,(keypoints['mouth_left']), 2, (0,155,255), 2)
                cv2.circle(frame,(keypoints['mouth_right']), 2, (0,155,255), 2)
    #display resulting frame
        if ret:
            cv2.imshow('Video', frame)
            out.write(frame)
            if cv2.waitKey(1) == 13: #13 은 엔터키
                break
        else:
            print("no frame")
            break
    out.release()
else:
    print("can't open file")
    # 얼굴에 직사각형 그리기
       

cap.release()
cv2.destroyAllWindows()

# (c) yolo를 이용한 동영상분석

In [ ]:
from imageai.Detection import VideoObjectDetection
import os
from matplotlib import pyplot as plt


execution_path = os.getcwd()

color_index = {'bus': 'red', 'handbag': 'steelblue', 'giraffe': 'orange', 'spoon': 'gray', 'cup': 'yellow', 'chair': 'green', 'elephant': 'pink', 'truck': 'indigo', 'motorcycle': 'azure', 'refrigerator': 'gold', 'keyboard': 'violet', 'cow': 'magenta', 'mouse': 'crimson', 'sports ball': 'raspberry', 'horse': 'maroon', 'cat': 'orchid', 'boat': 'slateblue', 'hot dog': 'navy', 'apple': 'cobalt', 'parking meter': 'aliceblue', 'sandwich': 'skyblue', 'skis': 'deepskyblue', 'microwave': 'peacock', 'knife': 'cadetblue', 'baseball bat': 'cyan', 'oven': 'lightcyan', 'carrot': 'coldgrey', 'scissors': 'seagreen', 'sheep': 'deepgreen', 'toothbrush': 'cobaltgreen', 'fire hydrant': 'limegreen', 'remote': 'forestgreen', 'bicycle': 'olivedrab', 'toilet': 'ivory', 'tv': 'khaki', 'skateboard': 'palegoldenrod', 'train': 'cornsilk', 'zebra': 'wheat', 'tie': 'burlywood', 'orange': 'melon', 'bird': 'bisque', 'dining table': 'chocolate', 'hair drier': 'sandybrown', 'cell phone': 'sienna', 'sink': 'coral', 'bench': 'salmon', 'bottle': 'brown', 'car': 'silver', 'bowl': 'maroon', 'tennis racket': 'palevilotered', 'airplane': 'lavenderblush', 'pizza': 'hotpink', 'umbrella': 'deeppink', 'bear': 'plum', 'fork': 'purple', 'laptop': 'indigo', 'vase': 'mediumpurple', 'baseball glove': 'slateblue', 'traffic light': 'mediumblue', 'bed': 'navy', 'broccoli': 'royalblue', 'backpack': 'slategray', 'snowboard': 'skyblue', 'kite': 'cadetblue', 'teddy bear': 'peacock', 'clock': 'lightcyan', 'wine glass': 'teal', 'frisbee': 'aquamarine', 'donut': 'mincream', 'suitcase': 'seagreen', 'dog': 'springgreen', 'banana': 'emeraldgreen', 'person': 'honeydew', 'surfboard': 'palegreen', 'cake': 'sapgreen', 'book': 'lawngreen', 'potted plant': 'greenyellow', 'toaster': 'ivory', 'stop sign': 'beige', 'couch': 'khaki'}


resized = False

def forFrame(frame_number, output_array, output_count, returned_frame):

    plt.clf()

    this_colors = []
    labels = []
    sizes = []

    counter = 0

    for eachItem in output_count:
        counter += 1
        labels.append(eachItem + " = " + str(output_count[eachItem]))
        sizes.append(output_count[eachItem])
        this_colors.append(color_index[eachItem])

    global resized

#     if (resized == False):
#         manager = plt.get_current_fig_manager()
#         manager.resize(width=1000, height=500)
#         resized = True

    plt.subplot(1, 2, 1)
    plt.title("Frame : " + str(frame_number))
    plt.axis("off")
    plt.imshow(returned_frame, interpolation="none")

    plt.subplot(1, 2, 2)
    plt.title("Analysis: " + str(frame_number))
    plt.pie(sizes, labels=labels, colors=this_colors, shadow=True, startangle=140, autopct="%1.1f%%")

    plt.pause(0.01)



video_detector = VideoObjectDetection()
video_detector.setModelTypeAsYOLOv3()
video_detector.setModelPath(os.path.join(execution_path, "models/yolo.h5"))
video_detector.loadModel()
plt.figure(figsize=(10,5))
plt.show()

video_detector.detectObjectsFromVideo(input_file_path=os.path.join(execution_path, "./videos/V2.mp4"), 
                                      output_file_path=os.path.join(execution_path, 'name') ,  
                                      frames_per_second=20, per_frame_function=forFrame,  
                                      minimum_percentage_probability=50, 
                                      return_detected_frame=True)